In [164]:
from lxml.etree import tostring
from itertools import chain
from lxml import etree
import urllib2
from pandas import DataFrame
import pandas
import time

In [142]:
# Trying to get info from one page

url = 'http://pyaterochka.ru/msk/actions/download/'
test = urllib2.urlopen(url)
page = test.read()
tree = etree.HTML(page)

In [143]:
def get_promo_territories(url):
    """
    This function get all Pyaterochka promo territories
    from main page.
    Requires urrlib2, pandas, lxml packages.
    """
    page_open = urllib2.urlopen(url)
    page_read = page_open.read()
    tree = etree.HTML(page_read.decode('utf-8'))

    li = []
    table = tree.xpath(u'//li/ul/li/a[@href = "#"]/..')

    for reg in table:
        temp_reg = etree.tostring(reg)
        temp_tree = etree.HTML(temp_reg)
        reg_name = temp_tree.xpath('//a[@href = "#"]/text()')

        city_arr = temp_tree.xpath('//ul/li/a')
        for city in city_arr:
            city_name = etree.HTML(etree.tostring(city)).xpath(u'//a/text()')
            city_href = etree.HTML(etree.tostring(city)).xpath(u'//a/@href')
            li.append([reg_name, city_name, city_href])

    df = DataFrame(li)
    #print df
    return df


In [144]:
def get_one_page(url):
    
    """
    This function get one Pyaterochka promo page and parse next information from it:
    bonus type, N + 1 parameter, old sku price, new sku price, sku title.
    It take URL on enter and return pandas data.frame.
    Requires pandas dataframe, urllib2, lxml
    """    

    page_open = urllib2.urlopen(url)
    page_read = page_open.read()
    tree = etree.HTML(page_read)
    
    li = []
    table = tree.xpath('//div[@class = "item"]')
    for x in table:
        temp =  etree.tostring(x)
        temp = temp.replace('<sup>', '.')
        temp = temp.replace('</sup>', '')
        temp_tree = etree.HTML(temp)
        newprice = ''.join(temp_tree.xpath('//div//span[@class = "new-price"]/text()')).replace('.',',')
        oldprice = ''.join(temp_tree.xpath('//div//div[@class = "old-price"]/div/text()')).replace('.',',')
        title = ''.join(temp_tree.xpath('//div//div[@class = "title"]/text()'))
        bonus = ''.join(temp_tree.xpath('//div//div[@class = "bonus"]/text()'))
        np1 = ''.join(temp_tree.xpath('//div//div[@class = "actrule"]/text()'))
        li.append([bonus, np1, oldprice, newprice, title])

    df = DataFrame(li)
    return df

In [146]:
terrs = get_promo_territories('http://pyaterochka.ru')
print terrs
#print get_one_page("http://pyaterochka.ru/msk/actions/download/")

                         0                     1                      2
0     [Московская область]              [Москва]                [/msk/]
1     [Московская область]            [Авсюнино]          [/avsunino_/]
2     [Московская область]           [Андреевка]          [/andreevka/]
3     [Московская область]           [Апрелевка]          [/aprelevka/]
4     [Московская область]            [Балашиха]         [/balashikha/]
5     [Московская область]            [Барыбино]           [/baribino/]
6     [Московская область]        [Белоозерский]       [/beloozerskiy/]
7     [Московская область]            [Белоомут]           [/beloomut/]
8     [Московская область]           [Березняки]         [/bereznyaki/]
9     [Московская область]         [Богородское]        [/bogorodskoe/]
10    [Московская область]      [Большие Вяземы]    [/bolshie_vyazemi/]
11    [Московская область]       [Большие Дворы]      [/bolshie_dvori/]
12    [Московская область]             [Брёхово]          [/bryo

In [158]:
def main(url):
    """
    Main function uses functions 'get_one_page' and 'get_promo_territories' to load 
    whole Pyaterochka booklet from Internet.
    """
    terrs = get_promo_territories(url)
    df = DataFrame()
    # в боевом режиме убрать [:] из строчи ниже
    for i in terrs.index[:5]:
        region = terrs.ix[i][0]
        city= terrs.ix[i][1]
        href = terrs.ix[i][2]
        url_use = url + ''.join(href) + "actions/download/"
        prices_data = get_one_page(url_use)
        prices_data['Region'] = ''.join(region) #pandas. .Series(''.join(region))
        prices_data['City'] = ''.join(city) #pandas.Series(''.join(city))
        df = df.append(prices_data)
        print i
    return df

    

        0    1       2       3  \
0    -35%        56,95   36,90   
1    -41%       514,00  299,00   
2    -40%       169,00   99,90   
3    -19%        64,25   51,90   
4                        99,90   
5    -46%       369,00  199,00   
6    -16%       209,00  175,00   
7    -22%       219,00  169,00   
8    -26%       189,00  139,00   
9   АКЦИЯ  1+1  255,00  127,50   
10   -30%       229,00  159,00   
11  АКЦИЯ  1+1  159,00   79,50   
12  АКЦИЯ  1+1  269,00  134,50   
13  АКЦИЯ  1+1  239,00  119,50   
14   -44%       289,00  159,00   
15   -29%       269,00  189,00   
16   -37%        71,95   44,90   
17   -16%        59,95   49,90   
18   -20%        59,95   47,90   
19  АКЦИЯ  1+1  109,00   54,50   
20   -13%        42,90   36,90   
21   -25%       199,00  149,00   
22   -37%       349,00  219,00   
23   -31%        57,95   39,90   
24   -23%        64,95   49,90   
25   -23%       105,00   79,90   
26   -25%        79,95   59,90   
27  АКЦИЯ  1+1  179,80   89,90   
28   -37%     

In [174]:
# get all promo prices
df_all = main("http://pyaterochka.ru")
df_all.columns = ['Sale','N + 1','Old price','New price','Name','Territory', 'City']

0
1
2
3
4


In [177]:
print str(time.strftime("%Y-%m-%d"))
#print df_all
adress = u'str(time.strftime("%Y-%m-%d")) + '_py.xlsx'
print adress
writer = pandas.ExcelWriter(adress) #, engine='xlsxwriter')
df_all.to_excel(writer, sheet_name = "Final")
writer.save()

#df_all.to_excel('')

2015-10-13
2015-10-13_py.xlsx
